# 1.Poolingレイヤのforwardの理解 

##  1-1. 実装コードを観察する
Poolingクラスの行数は多くないので、全体をのせる。

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from common.util import *

#from common/layers.py
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx


## 1-2. forwardでは何をやっているのか？概要をざっくりと理解する

(im2colを一度理解してしまえば、）概念理解的にはそれほど難解なことをやっていないようである。
テキストのＰ２２８〜Ｐ２２９の解説にあるとおり、以下になる。
1. プールフィルタのサイズ（高さ、幅）で、im2colする。結果は行ベクトルからなる行列ができる
2. 行ベクトル毎に最大の大きさとなる要素を取り出す　→　一本の列ベクトルが出来上がる
3. 列ベクトルを並び替えて、画像の枚数だけ行列をつくる

それでは、forwardの動作をサンプルを作りながらステップバイステップで確認していく。

## 1-3. 全体をサンプル作って試していく

In [2]:
from my_common.util import init_sample_matrix

### 全体のパラメータ
単純のため、strideを2、padを0に設定する(ch07/simple_convnet.pyの実装を見ると、strideは2で、padは0である)

In [3]:
#入力画像の高さ、幅、および、チャネル数。画像の枚数
H  = 4
W  = 5
C  = 1 #mnistはチャンネル数1のようなので、このサンプルテストもそれに合わせる(7.4.2と同じ)
N  = 2 #入力画像の枚数

#poolingレイヤの基本パラメータ
pool_h = 2
pool_w = 2
stride = 2
pad    = 0

#出力データ（画像）の高さと幅（自動的に計算される。）
out_h = int(1 + (H - pool_h) / stride)
out_w = int(1 + (W - pool_w) / stride)

print("出力画像の高さ=%d と 幅=%d" % (out_h ,out_w))

出力画像の高さ=2 と 幅=2


### 入力画像(x)を作る

In [4]:
#xの用意(mnistはチャンネル数1のようなので、このテストもそれに合わせる)
print("=== preparing of x===")
x1 = init_sample_matrix(filter_num = 1, channel=C, height=H, width=W) #filter番号(=画像番号)を識別する数値を与える(1,2~)
x2 = init_sample_matrix(filter_num = 2, channel=C, height=H, width=W)
x = np.array([[x1],[x2]])
print(x.shape)
print(x)

=== preparing of x===
(2, 1, 4, 5)
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]]]


### xに対してim2colを行う

In [5]:
col = im2col(x, pool_h, pool_w, stride, pad)
print(col)

[[1111. 1112. 1121. 1122.]
 [1113. 1114. 1123. 1124.]
 [1131. 1132. 1141. 1142.]
 [1133. 1134. 1143. 1144.]
 [2111. 2112. 2121. 2122.]
 [2113. 2114. 2123. 2124.]
 [2131. 2132. 2141. 2142.]
 [2133. 2134. 2143. 2144.]]


テキストのＰ２２８の説明にあるとおり、プーリングの大きさで行ベクトルが切り出されていることがわかる。
次にreshapeを行う。行は自動で決定して、列をpool_h * pool_wの大きさで指定する

In [6]:
col = col.reshape(-1, pool_h * pool_w)
print(col)

[[1111. 1112. 1121. 1122.]
 [1113. 1114. 1123. 1124.]
 [1131. 1132. 1141. 1142.]
 [1133. 1134. 1143. 1144.]
 [2111. 2112. 2121. 2122.]
 [2113. 2114. 2123. 2124.]
 [2131. 2132. 2141. 2142.]
 [2133. 2134. 2143. 2144.]]


結果は変わらなかった(おそらくこのreshapeは画像とチャンネル数が複数の時にＰ２２８の図で説明しているような並びにするためのものであると思う。一応、このnote bookの最後に参考として、複数枚数、チャンネルの場合の実行結果を乗せておく。)

### 要素の最大値を求める
np.argmax,np.maxにそれぞれaxis=1(行毎の演算)を仕掛ける。なお、argmaxはbackwardの計算時に使うもの。

In [7]:
arg_max = np.argmax(col, axis=1)
print(arg_max)
out = np.max(col, axis=1)
print(out)

[3 3 3 3 3 3 3 3]
[1122. 1124. 1142. 1144. 2122. 2124. 2142. 2144.]


### transposeを行い、結果を整形する
綺麗に、画像毎、チャネル毎にデータがまとまっている。

In [8]:
out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
print(out)

[[[[1122. 1124.]
   [1142. 1144.]]]


 [[[2122. 2124.]
   [2142. 2144.]]]]


## 1-4. [参考]  複数画像枚、複数チャンネルの場合

In [9]:
#入力画像の高さ、幅、および、チャネル数。画像の枚数
H  = 4 
W  = 5 
C  = 2 
N  = 2 #入力画像の枚数

#poolingレイヤの基本パラメータ
pool_h = 2
pool_w = 2
stride = 2
pad    = 0

#出力データ（画像）の高さと幅（自動的に計算される。）
out_h = int(1 + (H - pool_h) / stride)
out_w = int(1 + (W - pool_w) / stride)

print("出力画像の高さ=%d と 幅=%d" % (out_h ,out_w))

#xの用意
print("=== preparing of x===")
x11 = init_sample_matrix(filter_num = 1, channel=1, height=H, width=W) #filter番号(=画像番号)を識別する数値を与える(1,2~)
x12 = init_sample_matrix(filter_num = 1, channel=2, height=H, width=W)
x21 = init_sample_matrix(filter_num = 2, channel=1, height=H, width=W) #filter番号(=画像番号)を識別する数値を与える(1,2~)
x22 = init_sample_matrix(filter_num = 2, channel=2, height=H, width=W)
x = np.array([[x11,x12],[x21,x22]])
print(x.shape)
print(x)

出力画像の高さ=2 と 幅=2
=== preparing of x===
(2, 2, 4, 5)
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]

  [[1211 1212 1213 1214 1215]
   [1221 1222 1223 1224 1225]
   [1231 1232 1233 1234 1235]
   [1241 1242 1243 1244 1245]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]

  [[2211 2212 2213 2214 2215]
   [2221 2222 2223 2224 2225]
   [2231 2232 2233 2234 2235]
   [2241 2242 2243 2244 2245]]]]


In [10]:
col = im2col(x, pool_h, pool_w, stride, pad)
print(col)

[[1111. 1112. 1121. 1122. 1211. 1212. 1221. 1222.]
 [1113. 1114. 1123. 1124. 1213. 1214. 1223. 1224.]
 [1131. 1132. 1141. 1142. 1231. 1232. 1241. 1242.]
 [1133. 1134. 1143. 1144. 1233. 1234. 1243. 1244.]
 [2111. 2112. 2121. 2122. 2211. 2212. 2221. 2222.]
 [2113. 2114. 2123. 2124. 2213. 2214. 2223. 2224.]
 [2131. 2132. 2141. 2142. 2231. 2232. 2241. 2242.]
 [2133. 2134. 2143. 2144. 2233. 2234. 2243. 2244.]]


im2colしただけでは、複数チャネルのデータが同じ行ベクトルに入ってしまい、この後のnp.max(col,axis=1)で一発で最大値を計算できない。
このため、チャンネル毎に並べ替える必要がある。
以下の処理で並び替えができるらしい。
（numpyの自動指定の仕様を熟知しないと、この辺の処理の理由はわからないだろうが。。。）

In [11]:
col = col.reshape(-1, pool_h * pool_w)
print(col)

[[1111. 1112. 1121. 1122.]
 [1211. 1212. 1221. 1222.]
 [1113. 1114. 1123. 1124.]
 [1213. 1214. 1223. 1224.]
 [1131. 1132. 1141. 1142.]
 [1231. 1232. 1241. 1242.]
 [1133. 1134. 1143. 1144.]
 [1233. 1234. 1243. 1244.]
 [2111. 2112. 2121. 2122.]
 [2211. 2212. 2221. 2222.]
 [2113. 2114. 2123. 2124.]
 [2213. 2214. 2223. 2224.]
 [2131. 2132. 2141. 2142.]
 [2231. 2232. 2241. 2242.]
 [2133. 2134. 2143. 2144.]
 [2233. 2234. 2243. 2244.]]


In [12]:
arg_max = np.argmax(col, axis=1)
print(arg_max)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [13]:
out = np.max(col, axis=1)
print(out)

[1122. 1222. 1124. 1224. 1142. 1242. 1144. 1244. 2122. 2222. 2124. 2224.
 2142. 2242. 2144. 2244.]


In [14]:
print("はじめから、out.reshape(N,C,out_h, out_w)と指定してしまうと、以下のように、１つのチャネルに別のチャネルのデータが混じってしまう。上の結果を観察すると、同じチャンネルは飛び飛びで存在しているようなので、これを上手くまとめる必要がある")
print(out.reshape(N,C,out_h, out_w))

print("out.reshape(N,out_h, out_w,C)　おもむろにこのように指定してあげると。。。まぁ、結果は変わらん？")
print(out.reshape(N,out_h, out_w,C))

print("out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)、つまり、(N,out_h, out_w,C)　→　(N,C,out_h,out_w)")
print(out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2))

#out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
#print(out)

はじめから、out.reshape(N,C,out_h, out_w)と指定してしまうと、以下のように、１つのチャネルに別のチャネルのデータが混じってしまう。上の結果を観察すると、同じチャンネルは飛び飛びで存在しているようなので、これを上手くまとめる必要がある
[[[[1122. 1222.]
   [1124. 1224.]]

  [[1142. 1242.]
   [1144. 1244.]]]


 [[[2122. 2222.]
   [2124. 2224.]]

  [[2142. 2242.]
   [2144. 2244.]]]]
out.reshape(N,out_h, out_w,C)　おもむろにこのように指定してあげると。。。まぁ、結果は変わらん？
[[[[1122. 1222.]
   [1124. 1224.]]

  [[1142. 1242.]
   [1144. 1244.]]]


 [[[2122. 2222.]
   [2124. 2224.]]

  [[2142. 2242.]
   [2144. 2244.]]]]
out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)、つまり、(N,out_h, out_w,C)　→　(N,C,out_h,out_w)
[[[[1122. 1124.]
   [1142. 1144.]]

  [[1222. 1224.]
   [1242. 1244.]]]


 [[[2122. 2124.]
   [2142. 2144.]]

  [[2222. 2224.]
   [2242. 2244.]]]]


最後に画像毎、チャネル毎に整列される。
transposeは所見だとどの軸がどのように変換されたのかがわかりにくい。この辺の素朴な理解をmy_jupyer/numpy_excersiceにまとめた（理解の参考に）。このtransposeはたとえば、(0,0,0,1)→(0,1,0,0)に変換されるので、2列目を次のチャネルに移すような動作をしていることがわかる。このような変形になるため、チャネル毎に整列されることになるようだ。

transposeは添字がどのように変換されるか（例：３次元の場合、transpose(2,1,0)ならx(i,j,k)→x'(k,j.i)というふうに）１つずつ理解しながらどのように要素が移動するのかをトレースすると一発でわかる（素朴な方法だが。。。これしかないのかな？）

ただ、このtransposeの動作（というかＮ次元の転置の動作）を頭に叩き込んでおかないと、機械学習のアルゴリズムなんて考えられないだろうなぁ